# 6 Thomson-Reuters Tick History intraday data


In [1]:
import pandas as pd
import numpy as np

import re

import pdb

import glob

allfiles=glob.glob("data/raw/TRTH/equities/US/trade/SPY.P/*")

### Parallelization

In [2]:
import dask
dask.config.set(scheduler="processes")

@dask.delayed
def load_TRTH_trade(filename,
             tz_exchange="America/New_York",
             only_non_special_trades=True,
             only_regular_trading_hours=True,
             open_time="09:30:00",
             close_time="16:00:00",
             merge_sub_trades=True):
    try:
        if re.search('(csv|csv\\.gz)$',filename):
            DF = pd.read_csv(filename)
        if re.search('parquet$',filename):
            DF = pd.read_parquet(filename)

    except Exception as e:
        print("load_TRTH_trade could not load "+filename)
        print(e)
        return None
    
    if DF.shape[0]==0:
        return None
    
    if only_non_special_trades:
        DF = DF[DF["trade-stringflag"]=="uncategorized"]

    DF.drop(columns=["trade-rawflag","trade-stringflag"],axis=1,inplace=True)
    
    DF.index = pd.to_datetime(DF["xltime"],unit="d",origin="1899-12-30",utc=True)
    DF.index = DF.index.tz_convert(tz_exchange)  # .P stands for Arca, which is based at New York
    DF.drop(columns="xltime",inplace=True)
    
    if only_regular_trading_hours:
        DF=DF.between_time(open_time,close_time)    # warning: ever heard e.g. about Thanksgivings?
    
    if merge_sub_trades:
           DF=DF.groupby(DF.index).agg(trade_price=pd.NamedAgg(column='trade-price', aggfunc='mean'),
                                       trade_volume=pd.NamedAgg(column='trade-volume', aggfunc='sum'))
    
    return DF



@dask.delayed
def load_TRTH_bbo(filename,
             tz_exchange="America/New_York",
             only_regular_trading_hours=True):
    
    try:
        if re.search(r'(csv|csv\\.gz)$',filename):
            DF = pd.read_csv(filename)
        if re.search(r'parquet$',filename):
            DF = pd.read_parquet(filename) 
    except Exception as e:
        print("load_TRTH_bbo could not load "+filename)

    if DF.shape[0]==0:
        return None
    
        
    DF.index = pd.to_datetime(DF["xltime"],unit="d",origin="1899-12-30",utc=True)
    DF.index = DF.index.tz_convert(tz_exchange)  # .P stands for Arca, which is based at New York
    DF.drop(columns="xltime",inplace=True)
    
    if only_regular_trading_hours:
        DF=DF.between_time("09:30:00","16:00:00")    # ever heard about Thanksgivings?
        
    return DF

In [3]:
%time allpromises=[load_TRTH_trade(fn) for fn in allfiles]

CPU times: user 5.65 ms, sys: 4.5 ms, total: 10.1 ms
Wall time: 8.9 ms


Thus, it takes almost no time at all to create execution promises. Let us check that we really have promises:

In [4]:
allpromises[0]

Delayed('load_TRTH_trade-bb263721-9ff9-4232-a317-f3847bfe45dc')

To actually perform a computation, simply call the compute() function

In [5]:
allpromises[0].compute()

,trade_price,trade_volume
xltime,,
2009-04-03 09:30:00.649000192-04:00,83.500000,1000
2009-04-03 09:30:00.744000-04:00,83.490000,500
2009-04-03 09:30:00.933000192-04:00,83.505000,1257
2009-04-03 09:30:00.941000192-04:00,83.508125,7771
2009-04-03 09:30:01.777000192-04:00,83.516000,671
...,...,...
2009-04-03 15:59:59.867000064-04:00,84.260000,300
2009-04-03 15:59:59.876999936-04:00,84.260000,400
2009-04-03 15:59:59.930999808-04:00,84.260000,100


Now, let us load all the files in a parallel way

In [6]:
alldata=dask.compute(allpromises) 




There are alternative ways to delay a function: use dask.delayed(some_function) directly. 

In [7]:
allpromises=[dask.delayed(pd.read_csv)(fn) for fn in allfiles]

or defined a delayed version of a function

In [8]:
load_TRTH_trade_delayed=dask.delayed(load_TRTH_trade)

In [9]:
del alldata  # cleanup




 

### Merge trades and bbo data

In [10]:
trade_files=glob.glob("data/raw/TRTH/equities/US/trade/SPY.P/2009*")

allpromises=[load_TRTH_trade(fn) for fn in trade_files]
trades=dask.compute(allpromises)[0]

trades=pd.concat(trades)

In [11]:
bbo_files=glob.glob("data/raw/TRTH/equities/US/bbo/SPY.P/2009*")

allpromises=[load_TRTH_bbo(fn) for fn in bbo_files]
bbos=dask.compute(allpromises)[0]

bbos=pd.concat(bbos)

In [12]:
%time events=trades.join(bbos,how="outer")

CPU times: user 1min 41s, sys: 11.4 s, total: 1min 52s
Wall time: 1min 52s


In [13]:
del events   # otherwise 32Gb is not enough

In [14]:
trades.sort_index(inplace=True)
bbos.sort_index(inplace=True)

In [15]:
%time events=trades.join(bbos,how="outer")

CPU times: user 25.7 s, sys: 3.82 s, total: 29.5 s
Wall time: 29.5 s


In [16]:
events.shape

(89048311, 6)

We are entering into the realms of big data. Let us save this object

In [18]:
# before saving a parquet object, we need to ensure that the columns are in numeric format
events["bid-price"]=events["bid-price"].values.astype("float")
events["bid-volume"]=events["bid-volume"].values.astype("float")
events["ask-price"]=events["ask-price"].values.astype("float")
events["ask-volume"]=events["ask-volume"].values.astype("float")

#so far, one still needs to add the use_deprectated_int96_timestamps option
events.to_parquet("SPY_2009_events.parquet",use_deprecated_int96_timestamps=True,compression="brotli")

In [23]:
import vaex

ModuleNotFoundError: No module named 'vaex'

In [19]:
trades=vaex.open_mu(

SyntaxError: incomplete input (2086640168.py, line 1)